In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

#from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

trainfile = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv")
testfile = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_test.csv")
datapath = Path("../input/google-smartphone-decimeter-challenge")
truths = (datapath/'train').rglob('ground_truth.csv')
cols1 = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg',
       'lngDeg']
truthlist = []
for fpath in truths:
    fcsv = pd.read_csv(fpath, usecols = cols1)
    truthlist.append(fcsv)

truthdata = pd.concat(truthlist, ignore_index = True) # dump all truth files into a single file

In [ ]:

cols2 = ['collectionName','phoneName','millisSinceGpsEpoch','constellationType','svid','signalType','receivedSvTimeInGpsNanos','xSatPosM','ySatPosM','zSatPosM']
logs = (datapath/'train').rglob('*derived.csv')

logslist = []
for fpath in logs:
    logcsv = pd.read_csv(fpath, usecols = cols2)
    logslist.append(logcsv)

logsdata = pd.concat(logslist, ignore_index = True) # dump all derived logs files into a single file
logsdata['constellationType'].value_counts()

In [ ]:
logsdata['svid'].value_counts().index[0]

In [ ]:
logsdata_svtime = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).receivedSvTimeInGpsNanos.apply(lambda x: np.min(x)).reset_index()
logsdata_xsat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).xSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_ysat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).ySatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_zsat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).zSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_svid = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).svid.apply(lambda x: x.value_counts().index[0]).reset_index()
#logsdata_constellation = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).constellationType.apply(lambda x:x.value_counts().index[0]).reset_index()
logsdata_signaltype = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).signalType.apply(lambda x: x.value_counts().index[0]).reset_index()

logsdata1 = logsdata_svtime.merge(logsdata_xsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata2 = logsdata1.merge(logsdata_ysat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata3 = logsdata2.merge(logsdata_zsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata4 = logsdata3.merge(logsdata_svid, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata5 = logsdata4.merge(logsdata_constellation, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata_final = logsdata4.merge(logsdata_signaltype, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata_final['constellationType'] = logsdata_final.constellationType.apply(lambda x: "one" if x == 1 else("two" if x == 2 else("three" if x == 3 else("four" if x == 4 else("five" if x == 5 else("six" if x == 6 else "zero"))))))                                                                                                 
logsdata_final.head()

In [ ]:
print(len(logsdata_final))
print(len(trainfile))
traindata1 = trainfile.merge(truthdata, on = cols1[:3], suffixes=("_current","_truth")) # merge train file with observed data
traindata = pd.merge(traindata1, logsdata_final, how = 'left', on = ['collectionName','phoneName','millisSinceGpsEpoch'])
traindata.sort_values(by = ['phone','millisSinceGpsEpoch'])
#traindata = trainfile.merge(truthdata, on = cols1[:3], suffixes=("_current","_truth")) # merge train file with observed data
#traindata = traindata.merge(logsdata, on = cols1[:3]) # add columns from gnss logs to the traindata
traindata.head()
traindata = traindata.fillna(0)
traindata['latDeg_prev'] = traindata['latDeg_current'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['lngDeg_prev'] = traindata['lngDeg_current'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))    
traindata['latDeg_next'] = traindata['latDeg_current'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['lngDeg_next'] = traindata['lngDeg_current'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['time_prev'] = traindata['millisSinceGpsEpoch'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['time_next'] = traindata['millisSinceGpsEpoch'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))

#traindata['svid'] = pd.Categorical(traindata.svid)
#traindata['constellationType'] = pd.Categorical(traindata.constellationType) 
for i in traindata.index:
    if pd.isna(traindata['latDeg_prev'][i]):
        traindata['latDeg_prev'][i] = traindata['latDeg_current'][i]
        traindata['lngDeg_prev'][i] = traindata['lngDeg_current'][i]
        traindata['time_prev'][i] = traindata['millisSinceGpsEpoch'][i]
    if pd.isna(traindata['latDeg_next'][i]):
        traindata['latDeg_next'][i] = traindata['latDeg_current'][i]
        traindata['lngDeg_next'][i] = traindata['lngDeg_current'][i]
        traindata['time_next'][i] = traindata['millisSinceGpsEpoch'][i]
    
    #if traindata['time_prev'][i] == traindata['millisSinceGpsEpoch'][i]:
        #traindata['vel_lat'] = 0
        #traindata['vel_lng'] = 0
    #else:
        #traindata['vel_lat'] = (traindata['latDeg_current'][i] - traindata['latDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])
        #traindata['vel_lng'] = (traindata['lngDeg_current'][i] - traindata['lngDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])

traindata['time_since_last_read'] = traindata['millisSinceGpsEpoch'] - traindata['time_prev']
traindata['latd_prev_est'] = (traindata['latDeg_current'] - traindata['latDeg_prev'])*(10**6)
traindata['latd_next_est'] = (traindata['latDeg_next'] - traindata['latDeg_current'])*(10**6)
traindata['latd_prev_act'] = (traindata['latDeg_truth'] - traindata['latDeg_prev'])*(10**6)
traindata['lngd_prev_est'] = (traindata['lngDeg_current'] - traindata['lngDeg_prev'])*(10**6)
traindata['lngd_next_est'] = (traindata['lngDeg_next'] - traindata['lngDeg_current'])*(10**6)
traindata['lngd_prev_act'] = (traindata['lngDeg_truth'] - traindata['lngDeg_prev'])*(10**6)
print(len(traindata))
#print(len(trainfile))
traindata.head(1)

In [ ]:
traindata['lat_d'] = traindata['latDeg_current'] - traindata['latDeg_truth']
traindata['lng_d'] = traindata['lngDeg_current'] - traindata['lngDeg_truth']

In [ ]:
#traindata['time_since_last_read'].value_counts()
corr_df = pd.DataFrame(traindata.iloc[:, 3:].corr())# draw correlation between all columns from collection name, phone name, lat & lng Deg both estimated & observed
corr_df.to_csv('./corr_df.csv')
#testfile.head()

In [ ]:
x_cols = ['heightAboveWgs84EllipsoidM','time_since_last_read','xSatPosM','ySatPosM','zSatPosM','receivedSvTimeInGpsNanos','latd_prev_est','latd_next_est','lngd_prev_est','lngd_next_est']
xdata = traindata[x_cols] #creating the x variable df
xdata.head() # dummies for the categorical variable
#xdata_dummies = pd.get_dummies(xdata, prefix = "dum_")
#xdata_dummies.columns

In [ ]:
ydata = traindata[['latd_prev_act','lngd_prev_act']]
ydata.head()

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(xdata,ydata, test_size = 0.3, random_state = 162)
cols = list(xtrain.columns)
#latcols = [e for e in cols if e not in ('latd_prev_act','lngd_prev_act','millisSinceGpsEpoch')]
#lngcols = [e for e in cols if e not in ('latd_prev_act','lngd_prev_act','millisSinceGpsEpoch')]
#xtrain.head() -- x train & xtest is already transformed to have the right set of columns
ytrain_lat = ytrain[['latd_prev_act']]
ytest_lat = ytest[['latd_prev_act']]
ytrain_lng = ytrain[['lngd_prev_act']]
ytest_lng = ytest[['lngd_prev_act']]
xtrain.head()

In [ ]:
ytrain_lng.head() #check if datasets share the same indexes - for merging dataframes later

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV # library for random search hyper param tuning
from sklearn import metrics #library for metrics to measure model performance
param_grid = {#'bootstrap': [True],  ##applicable only for random forests not gradient boosting 
              'max_depth': [10,15],
              'n_estimators': [200,300]}   # create random grid
rf = GradientBoostingRegressor()
#rf_lat = RandomForestRegressor(bootstrap = True, n_estimators = 100, max_depth = 20, max_features = 'auto')
#rf_lat.fit(xtrain_lat, ytrain_lat)
#rf_lat.feature_importances_
rf_grid_lat = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, verbose = 150)
rf_grid_lat.fit(xtrain, ytrain_lat)
#xtrain_lat_arr = xtrain_lat.to_numpy()
#ytrain_lat_arr = ytrain_lat.to_numpy()
#ytrain_lat_arr
#rf_grid_lat = rf_grid.fit(xtrain_lat, ytrain_lat)
#rf_grid_lat.cv_results_
rf_grid_lat.best_params_ #throws the best model for latitude prediction

In [ ]:
rf_grid_lat.feature_importances_

In [ ]:
rf_grid_lat.cv_results_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
rf = GradientBoostingRegressor()
rf_lat = GradientBoostingRegressor(n_estimators = 200, max_depth = 10) # fitting with the best regressor & low variance
rf_lat.fit(xtrain, ytrain_lat)
ypred_lat = rf_lat.predict(xtest)
rf_lat.feature_importances_
#score(xtrain_lat, ytrain_lat)

In [ ]:
xtrain.columns

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {#'bootstrap': [True],  ##applicable only for random forests not gradient boosting 
              'max_depth': [15,20,25,30,35,40],
              'n_estimators': [300,400,500,600]} 
rf_grid_lng = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, cv = 5, n_iter = 8, verbose = 250)
rf_grid_lng.fit(xtrain, ytrain_lng)
rf_grid_lng.best_params_

In [ ]:
rf_grid_lng.cv_results_

In [ ]:
rf_lng = GradientBoostingRegressor(n_estimators = 300, max_depth = 20)
rf_lng.fit(xtrain_lng, ytrain_lng)
ypred_lng = rf_lng.predict(xtest_lng)

In [ ]:
#code fore adding phone to the test data
ytest_lat = ytest[['latDeg_truth','lngDeg_truth']]
#ytest_lat.head()
ytest_lat['millisSinceGpsEpoch'] = traindata['millisSinceGpsEpoch']
#ytest_lat.head()
ytest_lat['phone'] = traindata['phone']
ytest_lat.head()

In [ ]:
ytest_lat.reset_index(inplace = True)
ytest_lat = ytest_lat.drop(['index'], axis = 1)
#ytest_lat.head()

ytest_lat['latDeg_pred'] = pd.Series(ypred_lat)
ytest_lat['lngDeg_pred'] = pd.Series(ypred_lng)
ytest_lat.head()

In [ ]:
ytest_lat['lngDeg_truth'][5]

In [ ]:
#cacluating the havesine distance
from math import radians, cos, sin, asin, sqrt
def haversine(df):
    dist = []
    for i in df.index:
        lat1 = df['latDeg_truth'][i]
        lng1 = df['lngDeg_truth'][i]
        lat2 = df['latDeg_pred'][i]
        lng2 = df['lngDeg_pred'][i]
        # convert decimal degrees to radians 
        lat1,lng1,lat2,lng2 = map(radians, [lat1, lng1, lat2, lng2])

        # haversine formula 
        dlon = lng2 - lng1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
        d = c * r
        dist.append(d)
    return dist

In [ ]:
ytest_lat['dist'] = haversine(ytest_lat)
ytest_lat.head()

In [ ]:
ytest_lat.head()

In [ ]:
#xtest.head()
#xdata.head()
#final_df = pd.DataFrame()
ytest_pred_df = ytest_lat.groupby(['phone']).dist.apply(lambda x: np.percentile(x,50)).reset_index()
ytest_pred_df.head()
len(ytest_pred_df)

#df.groupby(['group'])['price'].apply(lambda x: np.percentile(x,60))
#ytest_lat.head()

In [ ]:
ytest_pred_df1 = ytest_lat.groupby(['phone']).dist.apply(lambda x: np.percentile(x,95)).reset_index()
ytest_pred_df1.head()

In [ ]:
ytest_pred_df['dist_95'] = ytest_pred_df1['dist']
avg_dist = []
for i in ytest_pred_df.index:
    a = (ytest_pred_df['dist'][i] + ytest_pred_df['dist_95'][i])/2
    avg_dist.append(a)
avg_dist
ytest_pred_df['avg_dist'] = avg_dist

In [ ]:
#ytest_pred_df = ytest_pred_df.drop(['avg_dist'], axis = 1)
ytest_pred_df.head()

In [ ]:
#final average
k = 0
s = 0
for i in ytest_pred_df.index:
    k = k + 1
    s = s + ytest_pred_df['avg_dist'][i]

l = s/k
print("overall average error across phones: %.5f", {l})

In [ ]:
submission = pd.read_csv("../input/google-smartphone-decimeter-challenge/sample_submission.csv")
submission.head()

In [ ]:
cols2 = ['collectionName','phoneName','millisSinceGpsEpoch','constellationType','svid','signalType','receivedSvTimeInGpsNanos','xSatPosM','ySatPosM','zSatPosM']
logs = (datapath/'test').rglob('*derived.csv')

logslist = []
for fpath in logs:
    logcsv = pd.read_csv(fpath, usecols = cols2)
    logslist.append(logcsv)

logsdata_test = pd.concat(logslist, ignore_index = True)

Add the additional columns from derived csv files

In [ ]:
logsdata_svtime = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).receivedSvTimeInGpsNanos.apply(lambda x: np.min(x)).reset_index()
logsdata_xsat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).xSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_ysat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).ySatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_zsat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).zSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_svid = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).svid.apply(lambda x: x.value_counts().index[0]).reset_index()
#logsdata_constellation = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).constellationType.apply(lambda x:x.value_counts().index[0]).reset_index()
logsdata_signaltype = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).signalType.apply(lambda x: x.value_counts().index[0]).reset_index()

logsdata1 = logsdata_svtime.merge(logsdata_xsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata2 = logsdata1.merge(logsdata_ysat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata3 = logsdata2.merge(logsdata_zsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata4 = logsdata3.merge(logsdata_svid, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata5 = logsdata4.merge(logsdata_constellation, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata_test_final = logsdata4.merge(logsdata_signaltype, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata_final['constellationType'] = logsdata_final.constellationType.apply(lambda x: "one" if x == 1 else("two" if x == 2 else("three" if x == 3 else("four" if x == 4 else("five" if x == 5 else("six" if x == 6 else "zero"))))))                                                                                                 
logsdata_test_final.head()

In [ ]:
len(xtrain_lat)

In [ ]:
#traindata = trainfile.merge(truthdata, on = cols1[:3], suffixes=("_current","_truth")) # merge train file with observed data
#testdata = testfile.merge(logsdata_test_final, on = cols1[:3]) # add columns from gnss logs to the traindata
testdata = pd.DataFrame()
print(len(testfile))
print(len(logsdata_test_final))
testdata = pd.merge(testfile, logsdata_test_final, how = 'left', on = ['collectionName','phoneName','millisSinceGpsEpoch'])
print(len(testdata))
testdata.tail()

In [ ]:
testdata['receivedSvTimeInGpsNanos'].value_counts()

In [ ]:
testdata = testdata.fillna(0)
testdata['svid'] = pd.Categorical(testdata.svid)
#testdata['constellationType'] = pd.Categorical(testdata.constellationType) 
testdata['signalType'] = pd.Categorical(testdata.signalType)
testdata.iloc[:, 3:].corr()

Create the columns needed for making prediction

In [ ]:
testdata.rename(columns = {'latDeg': 'latDeg_current', 'lngDeg': 'lngDeg_current'}, inplace = True)
testdata.sort_values(by = ['phone','millisSinceGpsEpoch'])
#testdata = testdata.fillna(0)
testdata['latDeg_prev'] = testdata['latDeg_current'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['lngDeg_prev'] = testdata['lngDeg_current'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))    
testdata['latDeg_next'] = testdata['latDeg_current'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['lngDeg_next'] = testdata['lngDeg_current'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['time_prev'] = testdata['millisSinceGpsEpoch'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['time_next'] = testdata['millisSinceGpsEpoch'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))

for i in testdata.index:
    if pd.isna(testdata['latDeg_prev'][i]):
        testdata['latDeg_prev'][i] = testdata['latDeg_current'][i]
        testdata['lngDeg_prev'][i] = testdata['lngDeg_current'][i]
        testdata['time_prev'][i] = testdata['millisSinceGpsEpoch'][i]
    if pd.isna(testdata['latDeg_next'][i]):
        testdata['latDeg_next'][i] = testdata['latDeg_current'][i]
        testdata['lngDeg_next'][i] = testdata['lngDeg_current'][i]
        testdata['time_next'][i] = testdata['millisSinceGpsEpoch'][i]
    
testdata['time_since_last_read'] = testdata['millisSinceGpsEpoch'] - testdata['time_prev']
print(len(testdata))
#print(len(trainfile))
testdata.head()

In [ ]:
testdata['receivedSvTimeInGpsNanos'].value_counts()

In [ ]:

#x_cols = ['latDeg_current','lngDeg_current','heightAboveWgs84EllipsoidM','latDeg_prev', 'latDeg_next', 'lngDeg_prev', 'lngDeg_next','time_since_last_read','xSatPosM','ySatPosM','zSatPosM','receivedSvTimeInGpsNanos']
#len(x_cols)
test_cols = [x for x in x_cols if x not in ('latDeg_truth','lngDeg_truth')]
xdata_test = testdata[test_cols] #creating the x variable df
#xdata_test.head() # dummies for the categorical variable
xdata_test_dummies = pd.get_dummies(xdata_test, prefix = "dum_")
xdata_test_dummies.columns

In [ ]:
xdata_test_dummies.head()
xdata_test_dummies = xdata_test_dummies.fillna(0)

In [ ]:
#arrange the columns correctly
col_list_train = [c for c in xtrain.columns]
xdata_test_dummies1 = pd.DataFrame()
for c in xtrain.columns:
    if c in xdata_test_dummies.columns:
        xdata_test_dummies1[c] = xdata_test_dummies[c]
    else:
        xdata_test_dummies1[c] = 0

xdata_test_dummies1.head()

In [ ]:
clen = len(xtrain.columns)
xtraincols = [c for c in xtrain.columns]
xtestcols = [c for c in xdata_test_dummies1.columns]
for i in range(clen):
    if xtraincols[i] != xtestcols[i]:
        print(xtestcols[i])
if len(xtraincols) != len(xtestcols):
    print("array len mismatch")

In [ ]:
xtrain_lat.columns

In [ ]:
xtest_lat.describe()

In [ ]:
cols = list(xdata_test_dummies1.columns)
latcols = [e for e in cols if e not in ('lngDeg_current','lngDeg_prev','lngDeg_next','latDeg_truth','lngDeg_truth')]
lngcols = [e for e in cols if e not in ('latDeg_current','latDeg_prev','latDeg_next','latDeg_truth','lngDeg_truth')]
xtest_lat = xdata_test_dummies[latcols]
xtest_lng = xdata_test_dummies[lngcols]
lat_pred = rf_lat.predict(xtest_lat)
xdata_test['latDeg'] = pd.Series(lat_pred)
lng_pred = rf_lng.predict(xtest_lng)
xdata_test['lngDeg'] = pd.Series(lng_pred)
xdata_test.head()

In [ ]:
xtest_lng.head()

In [ ]:
#testdata.head()
#testdata = testdata.rename({'latDeg': 'latDeg_current', 'lngDeg': 'lngDeg_current'}, axis=1)
#testdata.head()
testdata['latDeg'] = xdata_test['latDeg']
testdata['lngDeg'] = xdata_test['lngDeg']
collist_sub = ['phone','millisSinceGpsEpoch','latDeg','lngDeg']
subdata = testdata[collist_sub]
subdata.head()

In [ ]:
subdata.to_csv('./submissions_v3.csv', index = False)

In [ ]:
!kaggle competitions submit -c google-smartphone-decimeter-challenge -f 'submissions_v3.csv' -m "Message"